# KoBigBird KorSTS(Semantic Text Similarity) fine-tuning

긴 문장(최대 4096 토큰)이 입력 가능한 KoBigBird를 두 문장의 유사도 분석이 가능하도록 fine-tuning 합니다.

# 데이터셋(KorSTS Datasets)

카카오브레인에서 공개한 문장 유사도 데이터셋 KorSTS를 사용합니다.


다운로드 링크:
https://github.com/kakaobrain/KorNLUDatasets/tree/master/KorSTS

KorSTS 데이터셋은 두 문장 쌍과 두 문장이 유사한 정도를 나타내는 score(0~5점)가 label로 있는 데이터셋입니다.

BigBird에 두 문장 쌍을 입력하여 얻은 벡터 표현(문장 임베딩)으로 label score를 예측하도록 fine-tuning을 진행합니다.

## Preprocessing

In [1]:
data_path = './data/KorSTS/'

### train, dev, test data 읽어오기

리스트의 각 원소는 리스트로 저장됩니다. [[문장1, 문장2], score] 

In [2]:
import csv

train_data = []
dev_data = []
test_data = []

with open(data_path + 'sts-train.tsv') as file:
    tsv_file = csv.reader(file, delimiter="\t", quoting=csv.QUOTE_NONE)
    for line in tsv_file:
        train_data.append([[line[5], line[6]], line[4]])
        
with open(data_path + 'sts-dev.tsv') as file:
    tsv_file = csv.reader(file, delimiter="\t", quoting=csv.QUOTE_NONE)
    for line in tsv_file:
        dev_data.append([[line[5], line[6]], line[4]])
        
with open(data_path + 'sts-test.tsv') as file:
    tsv_file = csv.reader(file, delimiter="\t", quoting=csv.QUOTE_NONE)
    for line in tsv_file:
        test_data.append([[line[5], line[6]], line[4]])

In [3]:
train_data = train_data[1:]

In [4]:
dev_data = dev_data[1:]

In [5]:
test_data = test_data[1:]

In [6]:
print('train set 수:', len(train_data))
print('dev set 수:', len(dev_data))
print('test set 수:', len(test_data))

train set 수: 5749
dev set 수: 1500
test set 수: 1379


### label 스코어 0~1 사이의 값으로 정규화 하기

두 문장 벡터에 유사도를 계산할 때 코사인 유사도(-1 ~ +1)를 사용할 것이므로 0~5인 label score를 0과 1 사이의 값으로 정규화해야 합니다.

이를 위해 모든 데이터의 label socre를 5로 나누어줍니다. 

In [7]:
for tup in train_data:
    tup[1] = float(tup[1]) / 5

for tup in dev_data:
    tup[1] = float(tup[1]) / 5
    
for tup in test_data:
    tup[1] = float(tup[1]) / 5

In [8]:
print(train_data[0])
print(dev_data[0])
print(test_data[0])

[['비행기가 이륙하고 있다.', '비행기가 이륙하고 있다.'], 1.0]
[['안전모를 가진 한 남자가 춤을 추고 있다.', '안전모를 쓴 한 남자가 춤을 추고 있다.'], 1.0]
[['한 소녀가 머리를 스타일링하고 있다.', '한 소녀가 머리를 빗고 있다.'], 0.5]


## 학습 준비

### GPU or CPU

In [9]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"{device}를 사용합니다.")

/home/user03/anaconda3/envs/koelectra/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda를 사용합니다.


### KoBigBird 모델, 토크나이저 불러오기

In [10]:
from transformers import  AutoModel,AutoTokenizer

model = AutoModel.from_pretrained("monologg/kobigbird-bert-base", attention_type="original_full", add_pooling_layer=False)
tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base")

Some weights of the model checkpoint at monologg/kobigbird-bert-base were not used when initializing BigBirdModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'bert.pooler.bias', 'bert.pooler.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BigBirdModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 하이퍼파라미터

In [11]:
train_batch_size = 64
valid_bath_size = 32
epochs = 100
learning_rate=5e-5

## Dataset

데이터셋 클래스에 토크나이저를 받아와서 데이터셋 객체를 생성할 때 토크나이징 하여 리스트로 저장합니다.

In [12]:
from torch.utils.data import Dataset

class stsDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.sent1 = []
        self.sent2 = []
        self.label = []
        self.tokenizer = tokenizer
        for tup in train_data:
            self.sent1.append(self.tokenizer(tup[0][0], max_length=128, padding='max_length',return_tensors='pt'))
            self.sent2.append(self.tokenizer(tup[0][1], max_length=128, padding='max_length',return_tensors='pt'))
            self.label.append(tup[1])
        
    def __len__(self):
        return (len(self.label))
    
    def __getitem__(self, idx):
        return self.sent1[idx], self.sent2[idx], self.label[idx]

In [13]:
train_dataset = stsDataset(train_data, tokenizer)
valid_dataset = stsDataset(test_data, tokenizer)

## DataLoader

모델 훈련 시에 지정한 배치 크기만큼 데이터를 반환해주는 DataLodader입니다.

DataLoader는 Dataset객체를 인자로 받습니다.

In [14]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=train_batch_size
)

valid_dataloader = DataLoader(
    valid_dataset,
    shuffle=True,
    batch_size=valid_bath_size
)

## Train

모델 파라미터를 GPU로 이동합니다.

In [15]:
model.to(device)

BigBirdModel(
  (embeddings): BigBirdEmbeddings(
    (word_embeddings): Embedding(32500, 768, padding_idx=0)
    (position_embeddings): Embedding(4096, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BigBirdEncoder(
    (layer): ModuleList(
      (0): BigBirdLayer(
        (attention): BigBirdAttention(
          (self): BigBirdSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BigBirdSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inp

### 훈련에 필요한 optimizer, 코사인 유사도 함수 손실함수를 정의합니다.

optimizer에는 모델의 파라미터를 넘겨주고 학습률을 지정해줘야 합니다.

두 문장의 유사도를 코사인 유사도를 통해 구하면 -1 ~ +1 의 얻게 됩니다. 이 값이 최종 모델의 예측값이 됩니다.

BigBird를 통해 얻은 두 문장 벡터의 유사도를 구하기 위해 pytorch가 지원하는 코사인 유사도 함수를 사용합니다.

0과 1사이의 실수값을 예측하는 Task이므로 회귀 태스크라고 볼 수 있으며, 회귀에는 평균제곱오차(MSE loss)를 사용합니다.

모델의 예측값(두 문장 벡터의 코사인 유사도)과 label socre 의 차이가 0에 수렴하도록 학습됩니다.

In [16]:
from torch import nn

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
loss_fn = nn.MSELoss()

### 훈련 및 검증 

최저 valid loss를 갱신할 때마다 모델을 저장합니다. 현재 디렉토리의 model 폴더에 저장됩니다.

In [17]:
from tqdm import tqdm

In [18]:
best_val_loss = 100

for epoch in range(1, epochs+1):
    
    model.train()
    
    train_loss = []
    step = 0
    
    for i, batch in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        
        batch = tuple(t.to(device) for t in batch)
        b_sent1, b_sent2, b_label = batch 
        
        sent1_embs = model(b_sent1['input_ids'].squeeze(1), b_sent1['attention_mask'].squeeze(1), b_sent1['token_type_ids'].squeeze(1))[0][:,0,:]
        sent2_embs = model(b_sent2['input_ids'].squeeze(1), b_sent2['attention_mask'].squeeze(1), b_sent2['token_type_ids'].squeeze(1))[0][:,0,:]
        
        pred = cos(sent1_embs, sent2_embs)
        pred = pred.type(torch.float64).to(device)
        
        loss = loss_fn(pred, b_label)
        loss.backward()
        
        optimizer.step()
        
        train_loss.append(loss)
        step += 1
        
    print(f"train {epoch} epoch 종료  평균 loss: {sum(train_loss)/step}")
    
    with torch.no_grad():
        
        model.eval()

        valid_loss = []
        step = 0

        for i, batch in enumerate(tqdm(valid_dataloader)):
            
            batch = tuple(t.to(device) for t in batch)
            b_sent1, b_sent2, b_label = batch # [batch_size, max_len, 768]

            sent1_embs = model(b_sent1['input_ids'].squeeze(1), b_sent1['attention_mask'].squeeze(1), b_sent1['token_type_ids'].squeeze(1))[0][:,0,:]
            sent2_embs = model(b_sent2['input_ids'].squeeze(1), b_sent2['attention_mask'].squeeze(1), b_sent2['token_type_ids'].squeeze(1))[0][:,0,:]

            pred = cos(sent1_embs, sent2_embs)
            pred = pred.type(torch.float64).to(device)

            loss = loss_fn(pred, b_label)

            valid_loss.append(loss)
            step += 1
            
            
        val_loss = sum(valid_loss)/step
        print(f"vaild {epoch} epoch 종료  평균 loss: {val_loss}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            model_save_path = './model/' +  'saved_model_epoch_' + str(epoch) + '.pt'
            model.save_pretrained(model_save_path)

100%|█████████████████████████████████████████████████████████| 90/90 [01:04<00:00,  1.40it/s]


train 1 epoch 종료  평균 loss: 0.052405076862973576


100%|███████████████████████████████████████████████████████| 180/180 [00:21<00:00,  8.42it/s]


vaild 1 epoch 종료  평균 loss: 0.04134089954279399


100%|█████████████████████████████████████████████████████████| 90/90 [01:04<00:00,  1.40it/s]


...

vaild 99 epoch 종료  평균 loss: 0.005363745834323396


100%|█████████████████████████████████████████████████████████| 90/90 [01:04<00:00,  1.41it/s]


train 100 epoch 종료  평균 loss: 0.001214520034287132


100%|███████████████████████████████████████████████████████| 180/180 [00:21<00:00,  8.37it/s]

vaild 100 epoch 종료  평균 loss: 0.005338719528576356
